# Load Packages and Data

In [ ]:
!pip install pomegranate
!pip install Bio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 941.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 3.8 MB/s eta 0:00:00
  Created wheel for apricot-select: filename=apricot_select-0.6.1-py3-none-any.whl size=48764 sha256=a45321185470d853eb29981b8679934f9e43aef2a51e4a34e365c5803d751fc7
  Stored in directory: /root/.cache/pip/wheels/df/d8/f9/4d62b7272bff772a126a52e507212c2fd33c0b8416d9389665
Successfully built apricot-select
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.3 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/AvonYangXX1/DreamWalker.git

Cloning into 'DreamWalker'...
remote: Enumerating objects: 320, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 320 (delta 25), reused 86 (delta 13), pack-reused 211
Receiving objects: 100% (320/320), 891.33 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (37/37), done.
Updating files: 100% (128/128), done.


In [ ]:
from Bio.Align import PairwiseAligner
import pandas as pd
import numpy as np
import tensorflow as tf
# from sklearn.mixture import GaussianMixture
import pomegranate.distributions as pmg_dist
from pomegranate.gmm import GeneralMixtureModel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
class Preprocessing():
    def __init__(self, k_size=6):
        self.k_size = k_size
        kmers = self.ref_kmers("", self.k_size)
        self.vectorizer = CountVectorizer(vocabulary = kmers)
        self.seqs = []

    def ref_kmers(self, current_kmer, current_depth):
        if current_depth == 1:
            return [current_kmer+"a",current_kmer+"u",current_kmer+"c",current_kmer+"g"]
        else:
            ret = self.ref_kmers(current_kmer+"a",current_depth-1)
            for nt in ['u','c','g']:
                ret += self.ref_kmers(current_kmer+nt,current_depth-1)
            return ret

    def seq2kmer(self, seq, k):
        kmer = ""
        for i in range(0,len(seq)-k,1):
            kmer += seq[i:i+k]+" "
        return kmer[:-1]

    def CountKmers(self,seqs):
        if type(seqs) in [type([]),type(pd.core.series.Series([1]))]:
            kmer = pd.Series(seqs).apply(lambda x: self.seq2kmer(x, self.k_size))
            transformed_X = self.vectorizer.transform(kmer).toarray()
            return transformed_X
        else:
            raise ValueError("Invalid 'seqs' format. Expected formats are 'list' or 'pandas.core.series.Series'.")

    def ReadFASTA(self,filename,as_pd=True):
        if filename.split(".")[-1] not in ["fasta","fna","fa"]:
            raise ValueError('Invalid file format. Expected formats are ["fasta","fna","fa"].')
        file_handle = open(filename,"r")
        seqs = []
        seqid = []
        tmp_seq = ""
        for line in file_handle:
            if (line[0] == ">"):
                if tmp_seq != "":
                    seqs.append(tmp_seq)
                seqid.append(line.split("\n")[0][1:])
                tmp_seq = ""
            else:
                tmp_seq+=line.split("\n")[0]
        seqs.append(tmp_seq)
        file_handle.close()
        if as_pd:
            fasta = {}
            for i in range(len(seqs)):
                fasta[seqid[i]] = seqs[i]
            return pd.DataFrame(fasta,index=["sequence"]).transpose()["sequence"]
        else:
            return seqs, seqid

In [ ]:
def create_classifier():
    path = "DreamWalker/model_weights"
    vocab_size = 3982
    length = 19
    encoder_inputs = tf.keras.layers.Input(shape=(1024,))
    x = tf.keras.layers.RepeatVector(length, name="RepeatVector")(encoder_inputs)
    x = tf.keras.layers.GRU(1024, return_sequences=True, dropout=0.2, name="GRU0")(x)
    x = tf.keras.layers.GRU(1024, return_sequences=True, dropout=0.2, name="GRU1")(x)
    x = tf.keras.layers.Dense(vocab_size, activation="softmax")(x)
    classifier = tf.keras.models.Model(encoder_inputs, x)
    classifier.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
                       loss='sparse_categorical_crossentropy',metrics=["accuracy"])
    for i, layer in enumerate(classifier.layers):
        weights = np.load(f"{path}/ClassifierWeights/layer_{i}_weights.npz", allow_pickle=True)["weights"]
        layer.set_weights(weights)
        layer.trainable = False
    return classifier

def create_oracle(seq_len=40):
    inputs0 = tf.keras.layers.Input((seq_len, 43),name="SeqInput")
    inputs1 = tf.keras.layers.Input((1024,),name="BacteriaInput")
    # Extract Peptide Features
    x0 = tf.keras.layers.Conv1D(128, 5, activation='relu', name="Conv1D_0")(inputs0) # kernel_size=5 works well
    x0 = tf.keras.layers.Conv1D(128, 5, activation='relu', name="Conv1D_1")(x0) # Just two layers work better
    x0 = tf.keras.layers.Flatten(name="Flatten_CNN")(x0)
    x0 = tf.keras.layers.Dense(512, activation="relu", name="CNN_Dense0")(x0)

    # Target Marker Gene Representation
    classifier = create_classifier()
    MarkerRepresentModule = tf.keras.models.Model(classifier.layers[0].input,
                                                  classifier.layers[-1].output,
                                                  name="ClassifierModule")
    x1 = MarkerRepresentModule(inputs1)
    x1 = tf.keras.layers.Conv1D(128, 4, activation='relu', name="Conv1D_GRU0")(x1)
    x1 = tf.keras.layers.Flatten(name="Flatten_GRU")(x1)
    x1 = tf.keras.layers.Dense(512, activation="relu", name="GRU_DenseLast")(x1) # mimic the previous version

    # FCN
    x = tf.keras.layers.Concatenate(axis=1, name="Concat_FCN")([x0, x1])
    x = tf.keras.layers.Dense(1024, activation="relu", name="FCN_Dense0")(x)
    x = tf.keras.layers.LayerNormalization(name="LayerNorm_0")(x)
    x = tf.keras.layers.Dense(512, activation="relu", name="FCN_Dense1")(x)
    x = tf.keras.layers.LayerNormalization(name="LayerNorm_1")(x)
    x = tf.keras.layers.Dense(1, activation=tf.keras.activations.softplus, name="Output")(x)
    model = tf.keras.models.Model([inputs0, inputs1], x, name="Oracle")
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss=tf.keras.losses.MeanAbsoluteError(),
                  metrics=[tf.keras.metrics.R2Score()])
    return model

In [ ]:
oracle = create_oracle()
path = "DreamWalker/model_weights/OracleWeights"
for i, layer in enumerate(oracle.layers):
    if i == 3:
        continue
    param = layer.get_weights()
    if len(param) == 0:
        continue
    weights = np.load(f"{path}/layer_{i}_weights.npz")["weights"]
    biases = np.load(f"{path}/layer_{i}_biases.npz")["biases"]
    layer.set_weights([weights, biases])

In [ ]:
aa_vocal = np.load("DreamWalker/model_weights/PepTV_vocal.npy")
PepTV = tf.keras.layers.TextVectorization(standardize=None, split="character",
                                          output_sequence_length=40,
                                          output_mode="int", name="PepTextVectorizer")
PepTV.set_vocabulary(aa_vocal)

In [ ]:
def evaluate_pep(pep, query):
    enc_aln_pep = PepTV(pep).numpy()
    enc_aln_pep = tf.one_hot(enc_aln_pep, 43)
    query_kmers = pp.CountKmers([query])
    mic = oracle.predict([enc_aln_pep, np.repeat(query_kmers, 10, axis=0)], verbose=0)
    return mic

# Modeling

In [ ]:
pp = Preprocessing(k_size=5)
urgent_targets = pp.ReadFASTA('urgent_targets.fasta')

In [ ]:
path_to_targets = 'DreamWalker/data/processed_data/Baseline/targets.csv'
path_to_keys = 'DreamWalker/data/processed_data/Baseline/keys.csv'
path_to_pep = "DreamWalker/data/processed_data/Baseline/enc_aln_pep.npz"

In [ ]:
class KinsfolkProfileSampler():
    def __init__(self, path_to_targets, path_to_keys, path_to_pep, query, nlargest=10):
        self.aligner = PairwiseAligner()
        self.targets = pd.read_csv(path_to_targets)
        self.keys = pd.read_csv(path_to_keys)
        self.peptides = np.load(path_to_pep)['data']
        aa_vocal = np.load("DreamWalker/model_weights/PepTV_vocal.npy")
        self.pep_decoder = tf.keras.layers.StringLookup(vocabulary=[""] + aa_vocal[2:].tolist(), invert=True, oov_token='[UNK]')
        self.weights, self.selected_pep = self.findpep(query, nlargest)
        self.gmm = None

    def SelectParam(self, min_n_components=2, max_n_components=10):
        bics = {}
        n_components = min_n_components
        while n_components <= max_n_components:
            bic = self.create_gmm(n_components=n_components)
            print(f"{n_components} components; bic={int(bic)}")
            bics[n_components] = bic
            n_components += 1
        argmin = np.argmin(list(bics.values()))
        n_components = list(bics.keys())[argmin]
        print(f"The number of components is determined as {n_components}.")
        self.create_gmm(n_components=n_components)
        return bics

    def BIC(self, n_components):
        lnL = self.gmm.log_probability(self.selected_pep).numpy().sum()
        n_categories = self.selected_pep.max() + 1
        k = (n_categories-1) * n_components + n_components - 1
        n = self.selected_pep.shape[0]
        bic = -2*lnL + k*np.log(n)
        return bic

    def create_gmm(self, n_components):
        n_categories = self.selected_pep.max() + 1
        bic = np.nan
        while np.isnan(bic):
            dists = [pmg_dist.Categorical(n_categories=n_categories) for i in range(n_components)]
            self.gmm = GeneralMixtureModel(dists, verbose=False, max_iter=100)
            self.gmm.fit(self.selected_pep, sample_weight=self.weights)
            bic = self.BIC(n_components)
            if np.isnan(bic):
                print('Failed to converge. Repeat current iteration.')
        return bic

    def align(self, query):
        scores = self.targets['Marker'].apply(lambda x: self.aligner.align(x, query)[0].score)
        return scores

    def findkins(self, query, nlargest=10):
        scores = self.align(query)
        indices = scores.nlargest(nlargest).index.values
        kins = pd.concat([scores.nlargest(nlargest), self.targets["Target"][indices]], axis=1)
        return kins

    def findpep(self, query, nlargest=10):
        kins = self.findkins(query, nlargest)
        pepids = kins.merge(self.keys, on='Target')
        pepindices = pepids['PepID'].apply(lambda x: int(x.split('pep.')[1])).values
        scores = pepids.iloc[:,0]
        weights = np.exp(scores.values/100) / np.exp(scores.values/100).sum()
        return weights, self.peptides[pepindices]

    def Design(self, n_pep=10):
        return self.indices2seq(self.gmm.sample(n_pep).numpy())

    def indices2seq(self, indices):
        decoded_sequences = []
        chars_array = self.pep_decoder(indices).numpy().astype('str')
        decoded_sequences += ["".join(chars) for chars in chars_array]
        return decoded_sequences


In [ ]:
kps = KinsfolkProfileSampler(path_to_targets, path_to_keys, path_to_pep,
                             query=urgent_targets[2])

In [ ]:
bics = kps.SelectParam(2, 10)

2 components; bic=7715
3 components; bic=6898
4 components; bic=6968
5 components; bic=6359
6 components; bic=6985
7 components; bic=6232
8 components; bic=5607
9 components; bic=5997
10 components; bic=5020
The number of components is determined as 10.


In [ ]:
pep = kps.Design(10)

In [ ]:
evaluate_pep(pep, urgent_targets[2])

array([[5.0083566],
       [2.8947423],
       [2.845046 ],
       [5.663798 ],
       [4.9198623],
       [5.201288 ],
       [4.9465523],
       [5.1193376],
       [5.751469 ],
       [2.9416316]], dtype=float32)

In [ ]:
evaluate_pep(pep, urgent_targets[1])

array([[5.0566487],
       [3.6762202],
       [3.787516 ],
       [3.8285596],
       [2.9584646],
       [3.632147 ],
       [3.5904891],
       [2.433917 ],
       [5.476742 ],
       [3.1930895]], dtype=float32)

In [ ]:
evaluate_pep(pep, urgent_targets[0])

array([[4.2584863],
       [3.7051752],
       [4.79426  ],
       [4.4406276],
       [4.86472  ],
       [4.563173 ],
       [4.313587 ],
       [5.139053 ],
       [4.440681 ],
       [3.966105 ]], dtype=float32)